# Importing Libraries

In [ ]:
def part_2(lst,b,year_1,year_2,para,para_s):
    
    fs_final,cagr_final,ratios_final = screener_data_function(lst,b)    
    rename_columns(fs_final,cagr_final,ratios_final)
#     check_df = fs_final[(fs_final['Year']==year_1) | (fs_final['Year']==year_2)]
    
    z = list(fs_final['Company'].values)
    check_df['Industry'] = bse_ticker['Industry'].loc[z].values

    
    # Remove Percentage 
    remove_percentage_function(check_df)
    # Convert Columns to Integers
    column_type_convertor(check_df)
    check_df['Debt/Asset'] = (check_df['Borrowings']+check_df['Other_Liabilities'])/(check_df['Total Assets'])
    check_df['Other Income/Total Revenue'] = check_df['Other_Income']/check_df['Sales']
    check_df['Other Liabilities/Total Liabilities'] = check_df['Other_Liabilities']/check_df['Total Liabilities']
    check_df['Investment/Total Assets'] = check_df['Investments']/check_df['Total Assets']


In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from matplotlib import pyplot as plt
import ssl

import warnings
warnings.filterwarnings("ignore")

import riskfolio as rp

import time

# set the max columns to none
pd.set_option('display.max_columns', None)

# Inputs 

In [2]:
bse_ticker = pd.read_csv('Master.csv')
bse_ticker.set_index('NSE Ticker',inplace=True)
cd = pd.read_csv('closing_dataset.csv',index_col='Date',parse_dates=True)

para = {
'OPM':[True,12,'>'],
'CFO':[True,0,'>'],
'Net Cash Flow':[True,0,'>']
    }

para_s = {
    'Financing Margin %':[True,12,'>'],
    'Net_Profit':[True,0,'>'],
    'Net Cash Flow':[False,0,'>']
}

# Get the Closing Price Dataframe 

In [3]:
def closing_dataset_funtion(tickers,start,end):
    df = yf.download(tickers, start=start, end=end,progress=False,show_errors=False)['Close']
    return df
# closing_dataset = closing_dataset_funtion(tickers,start,end)

In [4]:
# close_index = list(bse_ticker.index)

# closing_dataset = closing_dataset_funtion(close_index,'2017-01-01',end='2022-05-01')

# closing_dataset.to_csv('closing_dataset.csv')

# RSI Dataset Function

In [5]:
def RSI_dataset_function(dataset,n2=14):
    
    close_price = dataset.copy()
    
    for i in close_price:
        a = pd.DataFrame(close_price[i])
        
        # Omtting null values
        a = a.dropna(axis=0)
        
        change = a.diff().dropna()
        change
        
        change_up = change.clip(lower=0)
        
        change_down = (-1 * change.clip(upper=0)).abs()
        

        avg_up = change_up.rolling(n2).mean()
    
        avg_down = change_down.rolling(n2).mean().abs()

        rs = avg_up/avg_down   # calculating rs
        
        rsi = 100-(100/(1+rs))  # calculating RSI
        rsi.dropna(inplace=True)
        close_price[i] = rsi
    
    return close_price
# RSI_dataset = RSI_dataset_function(closing_dataset)

# SMA Dataset Function

In [6]:
def SMA_dataset_function(dataset,n=5):
    close_price = dataset.copy()
    return close_price.rolling(n).mean()
# SMA_dataset = SMA_dataset_function(closing_dataset)

# Get Slope

In [7]:
### Function to get the Linear Regression Slopes
def regr(x,y):
    regr1 = LinearRegression()
    regr1.fit(x,y)
    return regr1.coef_

In [8]:
## This function takes the input as the dataset. It returns the Slope of each and every column(stocks) of the dataset given
#as input. And return it as a dataframe.

def get_Slope(dataset): ## Input would be the Dataset
    close_price = dataset.copy()
    index_list = []
    regr2 = []   ### Empty list to store the slope of the input dataset.
    y_hat = close_price ### Defining y_hat which is same as dataset.
    y_hat = y_hat.reset_index(drop=True)    ### Resetting the index so that date becomes a column.

    for i in y_hat.columns[1:]:   #### Iterating through the columns of y_hat.
        y = y_hat[i]   ### y is taken as the each data point of each and every column.
        y.dropna(inplace=True)
        if len(y)>0:
            x = np.array(y.index).reshape(-1, 1)   ##The x axis taken as sequence of numbers. 
            regr2.append(regr(x,y))   ## Appending the coefficients to the empty list.
            index_list.append(i)
    return pd.DataFrame(regr2, index = index_list)   ### Return the Dataframe and the index is set as column names.
# get_Slope(RSI_dataset[-40:-10]) ### Calling the function.

### RSI Divergence Function

In [9]:
def RSI_divergence_function(SMA_dataset,RSI_dataset):
    SMA_slope = get_Slope(SMA_dataset)
    SMA_slope.rename(columns={0:'SMA'},inplace=True)
    RSI_slope = get_Slope(RSI_dataset)
    RSI_slope.rename(columns={0:'RSI'},inplace=True)
    divergence = pd.concat([SMA_slope,RSI_slope],axis=1)
    divergence['divergence'] = True

    for i in range(len(divergence)):
        if not (divergence.iloc[i][0] < 0) & (divergence.iloc[i][1]>0):
            divergence.divergence.iloc[i] = False
    return divergence

### RSI breakout function

In [10]:
def RSI_breakout_function(new_RSI_dataset,breakout_RSI_dataset): 
    """
    input: two RSI dataset which is broken in two windows
    """
     # taking the last value of both the windows
    new_RSI_dataset = new_RSI_dataset[-1:]
    breakout_RSI_dataset = breakout_RSI_dataset[-1:]
    
    # checking the last value of the last window is lesser than the latest window
    RSI_breakout = pd.DataFrame(new_RSI_dataset.iloc[0] < breakout_RSI_dataset.iloc[0])
    
    # changing the name of the column
    RSI_breakout.rename(columns={0:'RSI_breakout'},inplace=True)
    
    # returning RSI_breakout
    return RSI_breakout

### Breakout

In [11]:
def breakout_function(dataset,breakout_window = 10,RSI_divergence_window = 30):
    close_price = dataset.copy()
#     print(dataset.shape)
    
    # tries to two dataset which is RSI and SMA
    RSI_dataset = RSI_dataset_function(close_price)
    SMA_dataset = SMA_dataset_function(close_price)
#     print(RSI_dataset.shape)
#     print(SMA_dataset.shape)
    
    
    # Breaking the Dataset into Two Windows 
    
    # RSI DIVERGENCE WINDOW
    new_RSI_dataset = RSI_dataset[-RSI_divergence_window-breakout_window:-breakout_window]
    new_SMA_dataset = SMA_dataset[-RSI_divergence_window-breakout_window:-breakout_window]
#     print(new_RSI_dataset.shape)
#     print(new_SMA_dataset.shape)

    # BREAKOUT WINDOW
    breakout_RSI_dataset = RSI_dataset[-breakout_window:]
    breakout_SMA_dataset = SMA_dataset[-breakout_window:]
#     print(breakout_RSI_dataset.shape)
#     print(breakout_SMA_dataset.shape)
    
    # Calculating RSI Divergence
    RSI_divergence = RSI_divergence_function(new_SMA_dataset,new_RSI_dataset)
    
    # Calculating Price Breakout
#     price_breakout = price_breakout_function(new_SMA_dataset,breakout_SMA_dataset)

    # Calculating RSI Breakout
#     RSI_breakout = RSI_breakout_function(new_RSI_dataset,breakout_RSI_dataset)

    
    # Calculating Breakout SMA Slope
    breakout_slope = get_Slope(breakout_SMA_dataset)
    breakout_slope.rename(columns={0:'SMA_breakout'},inplace=True)
    
    # Calculating Breakout SMA Slope
    breakout_RSI_slope = get_Slope(breakout_RSI_dataset)
    breakout_RSI_slope.rename(columns={0:'RSI_breakout'},inplace=True)

    
    # Concatenating all the Dataframes
    breakout = pd.concat([RSI_divergence,breakout_slope,breakout_RSI_slope],axis=1)
    
    # Adding a breakout column with 
    breakout['breakout'] = None
    
#     breakout
    for i in range(len(breakout)):
        if ((breakout.iloc[i][2] == True)and(breakout.iloc[i][3]>0)and breakout.iloc[i][4]>0):
#         if ((breakout.iloc[i][2] == True)and(breakout.iloc[i][3]>0)):            
#             print(i)
#             print(True)
            breakout.breakout[i] = True
    return breakout

# Screener.in Scrapping

In [12]:
def fs_concat(pnl,bs,cf):
    """
    This function takes in the pnl,bs,cf and concatenate them
    """
    # concatenating the tables 
    fs = pd.concat([pnl,bs,cf])
    
    # setting the primary column as index 
    fs.set_index('Unnamed: 0',inplace=True)
    fs = fs.T
    return fs 
# fs_concat(pnl,bs,cf)

In [13]:
def cagr_function(csg,cpg,spc,roe):
    """
    This function takes in the csg, cpg, spc and roe table and concatenate them
    """
    
    # setting primary columns as index
    csg.set_index('Compounded Sales Growth',inplace=True)
    cpg.set_index("Compounded Profit Growth",inplace=True)
    spc.set_index("Stock Price CAGR",inplace=True)
    roe.set_index("Return on Equity",inplace=True)

    # concatenating
    cagr = pd.concat([csg,cpg,spc,roe],axis=1)

    return cagr
# cagr_function(csg,cpg,spc,roe)

In [14]:
def screener_scrape_function(ticker):
    """
    This function takes in the BSE ticker for a company and returns a financial statements(pnl,bs,cf), CAGR and ratios for the respectice company
    1. takes in the tickers
    2. add the ticker to the screener.in url
    3. extract the tables from above tickers
    4. assign variable to the different tables
    5. concatenate fs,pnl,cf table into single fs (financial statement) table
    6. concatenate csg,cpg,spc,roe in cage table
    """
    
    # Scrapping the tables from the url
    ssl._create_default_https_context = ssl._create_unverified_context
    
    scraped = pd.read_html(f"https://www.screener.in/company/{ticker}")
    
    # assinging variables to different tables
    qr = scraped[0] # Quaterly results
    pnl = scraped[1] # profit and loss
    csg = scraped[2] # compounded sales growth
    cpg = scraped[3] # compunded profit growth 
    spc = scraped[4] # stock price CAGR
    roe = scraped[5] # return on Equity
    bs = scraped[6] # balance sheet 
    cf = scraped[7] # cash flow 
    ratios = scraped[8] # ratios 
#     shareholding = scraped[9] # shareholding pattern
    
    # concatenating pbl,bs,cf
    fs = fs_concat(pnl,bs,cf)
    
    # concatenating csg,cpg,spc,roe
    cagr = cagr_function(csg,cpg,spc,roe)
    
    # setting the first columns as index in ratios
    ratios.set_index('Unnamed: 0',inplace=True)
    
    return fs,cagr,ratios.T
# fs,cagr,ratios = screener_scrape_function(541988)

In [15]:
def screener_data_function(lst,bse_list):
    fs_final = pd.DataFrame()
    cagr_final = pd.DataFrame()
    ratios_final = pd.DataFrame()

    for i in range(len(bse_list)):
#         print(bse_list[i])
        if (bse_list[i] != 'BSE') &  (bse_list[i] != 'CDSL'):
            time.sleep(2)
            fs,cagr,ratios = screener_scrape_function(int(bse_list[i]))
            fs = fs.reset_index()
            fs['Company'] = lst[i]
            fs_final = pd.concat([fs_final,fs])

            cagr = cagr.reset_index()
            cagr['Company'] = lst[i]
            cagr_final = pd.concat([cagr_final,cagr])

            ratios = ratios.reset_index()
            ratios['Company'] = lst[i]
            ratios_final = pd.concat([ratios_final,ratios])
        else:
            time.sleep(3)
            fs,cagr,ratios = screener_scrape_function(str(bse_list[i])+'/consolidated/')
            fs = fs.reset_index()
            fs['Company'] = lst[i]
            fs_final = pd.concat([fs_final,fs])

            cagr = cagr.reset_index()
            cagr['Company'] = lst[i]
            cagr_final = pd.concat([cagr_final,cagr])

            ratios = ratios.reset_index()
            ratios['Company'] = lst[i]
            ratios_final = pd.concat([ratios_final,ratios])
            
        
    return fs_final,cagr_final,ratios_final
    

In [16]:
def rename_columns(fs_final,cagr_final,ratios_final):
    fs_rename ={
        'index':'Year',
        'OPM %':'OPM',
        'Cash from Operating Activity\xa0+':'CFO',
        'Cash from Investing Activity\xa0+':'CFI',
        'Cash from Financing Activity\xa0+':'CFF',
        'Other Assets\xa0+':'Other_Assets',
        'Sales\xa0+':'Sales',
        'Expenses\xa0+':'Expenses',
        'Other Income\xa0+':'Other_Income',
        'Share Capital\xa0+':'Share_Capital',
        'Other Liabilities\xa0+':'Other_Liabilities',
        'Fixed Assets\xa0+':'Fixed_Assets',
        'Net Profit': 'Net_Profit'
    }

    fs_final.rename(columns=fs_rename,inplace=True)

    cagr_rename = {
        'index':'Year',
        'Compounded Sales Growth.1':'Sales Growth',
        'Compounded Profit Growth.1':'Profit Growth',
        'Stock Price CAGR.1':'Stock Price Growth',
        'Return on Equity.1':'Return on Equity.1'

    }

    cagr_final.rename(columns=cagr_rename, inplace=True)

    ratios_rename = {
        'index':'Year'
    }

    ratios_final.rename(columns=ratios_rename,inplace=True)
# rename_columns()

In [17]:
def remove_percentage_function(check_df):
    percentage_list = ['Tax %','OPM','Dividend Payout %','Financing Margin %']
    for column in percentage_list:
        try:
#             print(column)
            for i in range(len(check_df[column])):


                if type(check_df[column].iloc[i]) == str:
                    if ',' in check_df[column].iloc[i]:
                        check_df[column].iloc[i] = check_df[column].iloc[i].replace(',','')
                    check_df[column].iloc[i] = check_df[column].iloc[i][:-1]
                else:
                    check_df[column].iloc[i] = 0 
        except:
            None
# remove_percentage_function(check_df)

In [18]:
def column_type_convertor(check_df):
    for i in check_df:
        if (i != 'Year') & (i!='Company') & (i!='Industry'):
            check_df[i] = pd.to_numeric(check_df[i])
# column_type_convertor(check_df)

In [19]:
    # para = {
    #     'OPM':[True,12,'>'],
    #     'CFO':[True,0,'>'],
    #     'Net Cash Flow':[True,0,'>']
    # }
    # para_s = {
    #     'Financing Margin %':[True,12,'>'],
    #     'Net_Profit':[True,0,'>'],
    #     'Net Cash Flow':[False,0,'>']
    # }
    def filter_stocks(para,para_s,check_df):
        df = check_df.copy()
        df = df[df['Industry']!= 'Financial Services']

        # Financial Service Comapny
        df_s = check_df.copy()
        df_s = df_s[df_s['Industry']== 'Financial Services']
        df_s

        filters = df.copy()
        filters_s = df_s.copy()


        for i in para:
            if para[i][0]:
                if para[i][2] == '>':
                    filters = df[(df[i]>=para[i][1])]
                elif para[i][2] == '<':
                    filters = df[(df[i]<=para[i][1])]
                else:
                    filters = df[(df[i]==para[i][1])]
            df = filters


        if df_s.shape[0]>0:
            try:
                for i in para_s:
                    if para_s[i][0]:
                        if para_s[i][2] == '>':
                            filters_s = df_s[(df_s[i]>=para_s[i][1])]
                        elif para[i][2] == '<':
                            filters_s = df_s[(df_s[i]<=para_s[i][1])]
                        else:
                            filters_s = df_s[(df_s[i]==para_s[i][1])]
                    df_s = filters_s
            except:
                for i in para:
                    if para[i][2] == '>':
                        filters = df_s[(df_s[i]>=para[i][1])]
                    elif para[i][2] == '<':
                        filters = df_s[(df_s[i]<=para[i][1])]
                    else:
                        filters = df_s[(df_s[i]==para[i][1])]
                    df_s = filters_s
    #     print(df)
    #     print(df_s.index)
    #     display(check_df,df_s,df)
        filtered_ticker = list(df['Company'].values) + list(df_s['Company'].values)
        return filtered_ticker

In [20]:
# year_1 = 'Dec 2019'
# year_2 = 'Mar 2020'
# check_df = fs_final[(fs_final['Year']==year_1) | (fs_final['Year']==year_2)]

In [21]:
# filtered_ticker = list(check_df[(check_df['CFO']>0)]['Company'].values)

# Get The Returns Dataframe 

In [22]:
def returns_dataset_funtion(filtered_ticker,closing_dataset):
    return_df = pd.DataFrame() # Creating a empty Df
    close_price = closing_dataset.copy() # Creating a copy of the closing dataset
#     close_price.reset_index(inplace=True)

    # for every company in filtered list. 
    for i in filtered_ticker:
        a = close_price[i] # get a column from the closing price dataset
        # Omtting null values
        a = a.dropna(axis=0) # omit all the nan values
        a = a.pct_change().dropna() # get a return and remove the first columns
        return_df[i] = a

    return return_df
# returns_dataset = returns_dataset_funtion(filtered_ticker,closing_dataset)

# HRP function

In [23]:
def weightshrp(df1) :
    port = rp.HCPortfolio(returns=df1)
    model='HRP'
    codependence = 'pearson'
    rm = 'MV'
    rf = 0
    linkage = 'single'
    max_k = 10
    leaf_order = True
    w = port.optimization(model=model,
                          codependence=codependence,
                          rm=rm,
                          rf=rf,
                          linkage=linkage,
                          max_k=max_k,
                          leaf_order=leaf_order)
    return(w)
# weightshrp(returns_dataset)

# Part 1  

In [24]:
# start = '2020-04-30'
# end = '2021-04-30'
# bse_ticker = pd.read_csv('Master.csv')
# bse_ticker.set_index('NSE Ticker',inplace=True)
# cd = pd.read_csv('closing_dataset.csv',index_col='Date',parse_dates=True)

# breakout_window=10
# RSI_divergence_window = 30


def part_1(dataset,bse_ticker,start,end,breakout_window=10,RSI_divergence_window=40):
    start_index = pd.date_range(start=start,end=end)
#     future_index = pd.date_range(start=future_start,end=future_end)
 
    # Part 1
    closing_dataset = dataset.loc[dataset.index.intersection(start_index)]
    
    RSI_dataset = RSI_dataset_function(closing_dataset)
    SMA_dataset = SMA_dataset_function(closing_dataset)
#     print("Part 1 Done")
    
    # Part 2 
    a = breakout_function(closing_dataset,breakout_window=breakout_window,RSI_divergence_window=RSI_divergence_window)
    
    breakout = a.loc[a['breakout']==True]
    z = list(breakout.index)
    breakout['Industry'] = bse_ticker['Industry'].loc[z].values
#     print(breakout.index)
    display(breakout) ######
    lst = list(breakout.index)
#     print(lst)
    b = list(bse_ticker.loc[lst]['BSE Ticker'].values)
#     print(b)
    return lst,b,closing_dataset
    

In [25]:
# lst,b,closing_dataset = part_1(cd,bse_ticker,start=start,end=end,breakout_window=breakout_window,RSI_divergence_window=RSI_divergence_window)

# Part 2 

In [26]:
# year_1 = 'Dec 2020'
# year_2 = 'Mar 2021'

# para = {
# 'OPM':[False,12,'>'],
# 'CFO':[True,0,'>'],
# 'Net Cash Flow':[False,0,'>']
#     }

# para_s = {
#     'Financing Margin %':[True,12,'>'],
#     'Net_Profit':[True,0,'>'],
#     'Net Cash Flow':[False,0,'>']
# }


def part_2(lst,b,year_1,year_2,para,para_s):
    fs_final,cagr_final,ratios_final = screener_data_function(lst,b)
    rename_columns(fs_final,cagr_final,ratios_final)

    check_df = fs_final[(fs_final['Year']==year_1) | (fs_final['Year']==year_2)]
    z = list(check_df['Company'].values)
    check_df['Industry'] = bse_ticker['Industry'].loc[z].values

    
    # Remove Percentage 
    remove_percentage_function(check_df)
#     display(check_df)    
    # Convert Columns to Integers
    column_type_convertor(check_df)
    

    
    filtered_ticker = filter_stocks(para,para_s,check_df)

    
    
    
    
#     filterd = check_df[(check_df['CFO']>CFO_amount)]
#     display(filterd)
#     filtered_ticker = list(filterd['Company'].values)
    display(check_df)
#     print(filtered_ticker)
    return filtered_ticker

In [27]:
# filtered_ticker = part_2(lst,b,year_1,year_2,para,para_s)
# filtered_ticker

# Part 4

In [28]:
def part_4(filtered_ticker,closing_dataset):
    if len(filtered_ticker)>1:
        returns_dataset = returns_dataset_funtion(filtered_ticker,closing_dataset)
        w = weightshrp(returns_dataset.dropna())
#         display(w)
    else:
        print('Only 1 Company Found')
        return pd.DataFrame()
    return w

In [29]:
# w = part_4(filtered_ticker,closing_dataset)
# w

# Part 5

In [30]:
# future_start = '2021-04-30'
# future_end= '2021-05-31'
# future_index = pd.date_range(start=future_start,end=future_end)
def part_5(dataset,future_start,future_end,w,filtered_ticker,m):
#     display(w)
    if w.shape[0] > 0:
        future_index = pd.date_range(start=future_start,end=future_end)
        future_price_dataset = dataset.loc[dataset.index.intersection(future_index)]
        future_price = future_price_dataset.get(filtered_ticker)
        po = np.asarray(future_price[:1])
        p1 = np.asarray(future_price[-1:])
#         m = 100000
        port = m * np.array(w)
        re = p1/po
        print(np.sum(re * np.asarray(port).T))
        return np.sum(re * np.asarray(port).T)
    else:
        print('Only 1 Company Found')
        return m
    
    # main(cd,bse_ticker,start=start,end=end,year_1=year_1,year_2=year_2,future_start=future_start,future_end=future_end,m=100000)

In [31]:
# part_5(cd,future_start,future_end,w)

In [32]:
def main(start,end,future_start,future_end,money):
    start = start
    end = end

    breakout_window=10
    RSI_divergence_window = 30


    lst,b,closing_dataset = part_1(cd,bse_ticker,start=start,end=end,breakout_window=breakout_window,RSI_divergence_window=RSI_divergence_window)

    year_1 = 'Dec 2020'
    year_2 = 'Mar 2021'

    if len(lst) == 0:
        print("RSI NOT FOUND")
    else:
        filtered_ticker = part_2(lst,b,year_1,year_2,para,para_s)

        w = part_4(filtered_ticker,closing_dataset)
        display(w)

        future_start = future_start
        future_end= future_end
        # future_index = pd.date_range(start=future_start,end=future_end)

        money = part_5(cd,future_start,future_end,w,filtered_ticker,m)
    return money

In [33]:
# start = '2020-07-30'
# end = '2021-07-30'
# future_start = end
# future_end= '2021-08-30'
# main(start,end,future_start,future_end)

In [34]:
start_list = pd.date_range(start='2020-04-01',end='2022-04-01',freq='M')
m = 100000
for i in range(0,len(start_list)):
    try:
        start = start_list[i]
        end = start_list[i+12]
        future_start = start_list[i+12]
        future_end= start_list[i+13]
        print('Start:',start)
        print("End:",end)
        print("Future Start",future_start)
        print("Future End",future_end)
        m = main(start,end,future_start,future_end,m)
        print("-------------------------------")
    except:
        None

Start: 2020-04-30 00:00:00
End: 2021-04-30 00:00:00
Future Start 2021-04-30 00:00:00
Future End 2021-05-31 00:00:00


,SMA,RSI,divergence,SMA_breakout,RSI_breakout,breakout,Industry
BAJAJ-AUTO.NS,-10.525820,0.772644,True,27.001581,1.628192,True,Automobile and Auto Components
BAJAJFINSV.NS,-21.269661,0.155133,True,95.758145,1.583615,True,Financial Services
HDFC.NS,-4.545881,0.444055,True,0.609216,0.131995,True,Financial Services
HIKAL.NS,-0.045259,0.849093,True,9.091878,1.162378,True,Healthcare
ICICIPRULI.NS,-1.516549,0.302719,True,9.926183,0.874729,True,Financial Services
ICIL.NS,-0.083132,0.094184,True,0.680303,0.119129,True,Textiles
INDIAMART.NS,-18.591793,0.498611,True,66.622202,0.826405,True,Consumer Services
JSL.NS,-0.069664,0.428593,True,1.453273,0.936948,True,Metals & Mining
JUBLPHARMA.NS,-3.210705,0.797402,True,7.940121,0.259677,True,Healthcare
JYOTHYLAB.NS,-0.320534,0.065439,True,1.089636,0.512627,True,Fast Moving Consumer Goods


Unnamed: 0,Year,Sales,Expenses,Operating Profit,OPM,Other_Income,Interest,Depreciation,Profit before tax,Tax %,Net_Profit,EPS in Rs,Dividend Payout %,Share_Capital,Reserves,Borrowings,Other_Liabilities,Total Liabilities,Fixed_Assets,CWIP,Investments,Other_Assets,Total Assets,CFO,CFI,CFF,Net Cash Flow,Company,Revenue,Financing Profit,Financing Margin %,Industry
10,Mar 2021,27741.0,22800.00,4942.00,18.0,1263.00,7.00,259.0,5939.00,23.0,4555.00,157.40,89.0,289.00,24913.00,121,6206.00,31530.00,1665.00,16,22631.00,7218.00,31530.00,3114.00,-2867.00,-20,228.00,BAJAJ-AUTO.NS,NaN,NaN,0,Automobile and Auto Components
10,Mar 2021,392.0,142.00,250.00,64.0,-0.00,0.00,5.0,245.00,27.0,179.00,11.23,27.0,80.00,3721.00,0,68.00,3869.00,159.00,2,3495.00,213.00,3869.00,169.00,-149.00,0,20.00,BAJAJFINSV.NS,NaN,NaN,0,Financial Services
10,Mar 2021,NaN,4587.00,NaN,0.0,26.00,28615.00,159.0,14815.00,19.0,12027.00,66.67,34.0,361.00,108115.00,441365,17758.00,567599.00,2196.00,0,68637.00,496765.00,567599.00,-28473.00,-8500.00,34601,-2372.00,HDFC.NS,48150.0,14948.0,31,Financial Services
11,Mar 2021,1720.0,1398.00,323.00,19.0,5.00,36.00,85.0,206.00,36.0,133.00,10.80,19.0,25.00,909.00,610,370.00,1913.00,713.00,254,1.00,946.00,1913.00,229.00,-156.00,-97,-24.00,HIKAL.NS,NaN,NaN,0,Healthcare
10,Mar 2021,83180.0,83509.00,-330.00,-0.0,1668.00,56.00,60.0,1223.00,22.0,960.00,6.69,30.0,1436.00,7683.00,1200,206909.00,217228.00,435.00,22,212212.00,4559.00,217228.00,7320.00,-5090.00,1205,3434.00,ICICIPRULI.NS,NaN,NaN,0,Financial Services
11,Mar 2021,2515.0,2131.00,384.00,15.0,36.00,27.00,40.0,353.00,26.0,260.00,13.18,11.0,39.00,1235.00,541,431.00,2247.00,535.00,8,192.00,1512.00,2247.00,27.00,-196.00,158,-11.00,ICIL.NS,NaN,NaN,0,Textiles
10,Mar 2021,665.0,332.00,333.00,50.0,85.00,7.00,16.0,396.00,28.0,287.00,94.43,16.0,30.00,1586.00,63,835.00,2516.00,65.00,0,2267.00,184.00,2516.00,326.00,-1343.00,1038,22.00,INDIAMART.NS,NaN,NaN,0,Consumer Services
10,Mar 2021,11679.0,10283.00,1396.00,12.0,142.00,464.00,373.0,700.00,39.0,428.00,8.78,0.0,97.00,3002.00,2899,4266.00,10265.00,5679.00,55,491.00,4040.00,10265.00,1295.00,-154.00,-1107,34.00,JSL.NS,NaN,NaN,0,Metals & Mining
11,Mar 2021,2710.0,2282.00,427.00,16.0,48.00,102.00,100.0,273.00,21.0,214.00,13.46,37.0,16.00,1267.00,504,58.00,1845.00,127.00,0,1646.00,71.00,1845.00,608.00,117.00,-782,-56.00,JUBLPHARMA.NS,NaN,NaN,0,Healthcare
11,Mar 2021,1885.0,1568.00,317.00,17.0,-4.00,12.00,78.0,223.00,15.0,190.00,5.18,77.0,37.00,897.00,50,440.00,1424.00,457.00,10,141.00,815.00,1424.00,386.00,-127.00,-196,63.00,JYOTHYLAB.NS,NaN,NaN,0,Fast Moving Consumer Goods


,weights
BAJAJ-AUTO.NS,0.104867
INDIAMART.NS,0.044752
JSL.NS,0.033076
JYOTHYLAB.NS,0.126875
MCDOWELL-N.NS,0.122886
NAUKRI.NS,0.074764
PNCINFRA.NS,0.060954
SHILPAMED.NS,0.054286
SKFINDIA.NS,0.103589
SUNTV.NS,0.067927


112269.58354916045
-------------------------------
Start: 2020-05-31 00:00:00
End: 2021-05-31 00:00:00
Future Start 2021-05-31 00:00:00
Future End 2021-06-30 00:00:00


...

,SMA,RSI,divergence,SMA_breakout,RSI_breakout,breakout,Industry
SOLARINDS.NS,-0.551594,0.97455,True,5.735811,1.522547,True,Chemicals


Unnamed: 0,Year,Sales,Expenses,Operating Profit,OPM,Other_Income,Interest,Depreciation,Profit before tax,Tax %,Net_Profit,EPS in Rs,Dividend Payout %,Share_Capital,Reserves,Borrowings,Other_Liabilities,Total Liabilities,Fixed_Assets,CWIP,Investments,Other_Assets,Total Assets,CFO,CFI,CFF,Net Cash Flow,Company,Industry
10,Mar 2021,1584,1312,272,17,26,7,38,253,25,189,20.89,29,18,1139,125,318,1600,558,130,129,784,1600,148,-99,-66,-18,SOLARINDS.NS,Chemicals


Only 1 Company Found


""


Only 1 Company Found
-------------------------------


In [35]:
start_list = pd.date_range(start='2020-04-01',end='2022-04-01',freq='M')
m = 100000
for i in range(0,len(start_list),2):
    try:
        start = start_list[i]
        end = start_list[i+12]
        future_start = start_list[i+12]
        future_end= start_list[i+14]
        print('Start:',start)
        print("End:",end)
        print("Future Start",future_start)
        print("Future End",future_end)
        m = main(start,end,future_start,future_end,m)
        print("-------------------------------")
    except:
        None

Start: 2020-04-30 00:00:00
End: 2021-04-30 00:00:00
Future Start 2021-04-30 00:00:00
Future End 2021-06-30 00:00:00


,SMA,RSI,divergence,SMA_breakout,RSI_breakout,breakout,Industry
BAJAJ-AUTO.NS,-10.525820,0.772644,True,27.001581,1.628192,True,Automobile and Auto Components
BAJAJFINSV.NS,-21.269661,0.155133,True,95.758145,1.583615,True,Financial Services
HDFC.NS,-4.545881,0.444055,True,0.609216,0.131995,True,Financial Services
HIKAL.NS,-0.045259,0.849093,True,9.091878,1.162378,True,Healthcare
ICICIPRULI.NS,-1.516549,0.302719,True,9.926183,0.874729,True,Financial Services
ICIL.NS,-0.083132,0.094184,True,0.680303,0.119129,True,Textiles
INDIAMART.NS,-18.591793,0.498611,True,66.622202,0.826405,True,Consumer Services
JSL.NS,-0.069664,0.428593,True,1.453273,0.936948,True,Metals & Mining
JUBLPHARMA.NS,-3.210705,0.797402,True,7.940121,0.259677,True,Healthcare
JYOTHYLAB.NS,-0.320534,0.065439,True,1.089636,0.512627,True,Fast Moving Consumer Goods


Unnamed: 0,Year,Sales,Expenses,Operating Profit,OPM,Other_Income,Interest,Depreciation,Profit before tax,Tax %,Net_Profit,EPS in Rs,Dividend Payout %,Share_Capital,Reserves,Borrowings,Other_Liabilities,Total Liabilities,Fixed_Assets,CWIP,Investments,Other_Assets,Total Assets,CFO,CFI,CFF,Net Cash Flow,Company,Revenue,Financing Profit,Financing Margin %,Industry
10,Mar 2021,27741.0,22800.00,4942.00,18.0,1263.00,7.00,259.0,5939.00,23.0,4555.00,157.40,89.0,289.00,24913.00,121,6206.00,31530.00,1665.00,16,22631.00,7218.00,31530.00,3114.00,-2867.00,-20,228.00,BAJAJ-AUTO.NS,NaN,NaN,0,Automobile and Auto Components
10,Mar 2021,392.0,142.00,250.00,64.0,-0.00,0.00,5.0,245.00,27.0,179.00,11.23,27.0,80.00,3721.00,0,68.00,3869.00,159.00,2,3495.00,213.00,3869.00,169.00,-149.00,0,20.00,BAJAJFINSV.NS,NaN,NaN,0,Financial Services
10,Mar 2021,NaN,4587.00,NaN,0.0,26.00,28615.00,159.0,14815.00,19.0,12027.00,66.67,34.0,361.00,108115.00,441365,17758.00,567599.00,2196.00,0,68637.00,496765.00,567599.00,-28473.00,-8500.00,34601,-2372.00,HDFC.NS,48150.0,14948.0,31,Financial Services
11,Mar 2021,1720.0,1398.00,323.00,19.0,5.00,36.00,85.0,206.00,36.0,133.00,10.80,19.0,25.00,909.00,610,370.00,1913.00,713.00,254,1.00,946.00,1913.00,229.00,-156.00,-97,-24.00,HIKAL.NS,NaN,NaN,0,Healthcare
10,Mar 2021,83180.0,83509.00,-330.00,-0.0,1668.00,56.00,60.0,1223.00,22.0,960.00,6.69,30.0,1436.00,7683.00,1200,206909.00,217228.00,435.00,22,212212.00,4559.00,217228.00,7320.00,-5090.00,1205,3434.00,ICICIPRULI.NS,NaN,NaN,0,Financial Services
11,Mar 2021,2515.0,2131.00,384.00,15.0,36.00,27.00,40.0,353.00,26.0,260.00,13.18,11.0,39.00,1235.00,541,431.00,2247.00,535.00,8,192.00,1512.00,2247.00,27.00,-196.00,158,-11.00,ICIL.NS,NaN,NaN,0,Textiles
10,Mar 2021,665.0,332.00,333.00,50.0,85.00,7.00,16.0,396.00,28.0,287.00,94.43,16.0,30.00,1586.00,63,835.00,2516.00,65.00,0,2267.00,184.00,2516.00,326.00,-1343.00,1038,22.00,INDIAMART.NS,NaN,NaN,0,Consumer Services
10,Mar 2021,11679.0,10283.00,1396.00,12.0,142.00,464.00,373.0,700.00,39.0,428.00,8.78,0.0,97.00,3002.00,2899,4266.00,10265.00,5679.00,55,491.00,4040.00,10265.00,1295.00,-154.00,-1107,34.00,JSL.NS,NaN,NaN,0,Metals & Mining
11,Mar 2021,2710.0,2282.00,427.00,16.0,48.00,102.00,100.0,273.00,21.0,214.00,13.46,37.0,16.00,1267.00,504,58.00,1845.00,127.00,0,1646.00,71.00,1845.00,608.00,117.00,-782,-56.00,JUBLPHARMA.NS,NaN,NaN,0,Healthcare
11,Mar 2021,1885.0,1568.00,317.00,17.0,-4.00,12.00,78.0,223.00,15.0,190.00,5.18,77.0,37.00,897.00,50,440.00,1424.00,457.00,10,141.00,815.00,1424.00,386.00,-127.00,-196,63.00,JYOTHYLAB.NS,NaN,NaN,0,Fast Moving Consumer Goods


,weights
BAJAJ-AUTO.NS,0.104867
INDIAMART.NS,0.044752
JSL.NS,0.033076
JYOTHYLAB.NS,0.126875
MCDOWELL-N.NS,0.122886
NAUKRI.NS,0.074764
PNCINFRA.NS,0.060954
SHILPAMED.NS,0.054286
SKFINDIA.NS,0.103589
SUNTV.NS,0.067927


116609.43776826125
-------------------------------
Start: 2020-06-30 00:00:00
End: 2021-06-30 00:00:00
Future Start 2021-06-30 00:00:00
Future End 2021-08-31 00:00:00


...

RSI NOT FOUND
-------------------------------
Start: 2020-12-31 00:00:00
End: 2021-12-31 00:00:00
Future Start 2021-12-31 00:00:00
Future End 2022-02-28 00:00:00


,SMA,RSI,divergence,SMA_breakout,RSI_breakout,breakout,Industry
AJANTPHARM.NS,-2.933158,0.174115,True,15.549689,2.200176,True,Healthcare
ALKEM.NS,-7.059946,0.888943,True,14.027697,4.278671,True,Healthcare
ALKYLAMINE.NS,-6.758638,0.842334,True,18.712430,2.907538,True,Chemicals
AMARAJABAT.NS,-3.024053,0.227562,True,0.369453,0.640789,True,Automobile and Auto Components
ANGELONE.NS,-4.100121,0.476869,True,2.989939,3.222626,True,Financial Services
ATUL.NS,-1.575415,1.140951,True,36.955496,1.508210,True,Chemicals
BALKRISIND.NS,-10.721145,0.346638,True,6.500478,1.840663,True,Automobile and Auto Components
BALRAMCHIN.NS,-0.318487,0.289196,True,3.905757,2.886840,True,Fast Moving Consumer Goods
BASF.NS,-10.664111,1.838847,True,12.152488,1.040336,True,Chemicals
BRITANNIA.NS,-4.956455,0.232910,True,3.557212,0.257907,True,Fast Moving Consumer Goods


Unnamed: 0,Year,Sales,Expenses,Operating Profit,OPM,Other_Income,Interest,Depreciation,Profit before tax,Tax %,Net_Profit,EPS in Rs,Dividend Payout %,Share_Capital,Reserves,Borrowings,Other_Liabilities,Total Liabilities,Fixed_Assets,CWIP,Investments,Other_Assets,Total Assets,CFO,CFI,CFF,Net Cash Flow,Company,Industry
11,Mar 2021,2719,1800,919,34,102,7,111,902,25,676,78.08,12,17,2868,26,621,3532,1519,108,157,1748,3532,467,-163,-269,35,AJANTPHARM.NS,Healthcare
11,Mar 2021,7220,5268,1951,27,175,43,199,1884,11,1685,140.93,21,24,7602,1339,1808,10773,1758,323,2152,6539,10773,1324,-1121,-209,-5,ALKEM.NS,Healthcare
11,Mar 2021,1242,812,430,35,6,6,29,401,26,295,57.91,17,10,782,48,306,1145,458,138,30,519,1145,364,-213,-86,65,ALKYLAMINE.NS,Chemicals
11,Mar 2021,7150,6033,1117,16,86,11,319,873,26,647,37.87,29,17,4193,92,1494,5797,2455,399,281,2662,5797,802,-616,-122,64,AMARAJABAT.NS,Automobile and Auto Components
5,Mar 2021,1280,830,450,35,8,43,17,398,27,290,35.49,36,82,1020,1171,2509,4782,100,0,83,4599,4782,-912,2,896,-14,ANGELONE.NS,Financial Services
10,Mar 2021,3512,2665,847,24,103,2,120,828,24,631,213.23,9,30,3682,0,876,4587,1059,239,1551,1739,4587,677,-603,-64,10,ATUL.NS,Chemicals
11,Mar 2021,5758,3971,1786,31,161,10,406,1531,25,1155,59.77,28,39,5969,894,1139,8040,3334,856,1418,2432,8040,1363,-1158,-194,11,BALKRISIND.NS,Automobile and Auto Components
11,Mar 2021,4812,4098,714,15,33,39,112,596,21,470,22.37,11,21,2543,1240,758,4562,1599,14,177,2772,4562,649,-81,-569,-1,BALRAMCHIN.NS,Fast Moving Consumer Goods
11,Mar 2021,9558,8926,633,7,419,38,174,840,34,553,127.67,8,43,1743,285,3073,5144,817,53,0,4274,5144,483,182,-592,73,BASF.NS,Chemicals
10,Mar 2021,12379,10028,2351,19,293,98,167,2379,26,1760,73.07,216,24,3295,1798,2298,7416,1404,112,2950,2950,7416,1778,501,-2223,56,BRITANNIA.NS,Fast Moving Consumer Goods


,weights
AJANTPHARM.NS,0.049071
ALKYLAMINE.NS,0.008097
AMARAJABAT.NS,0.041641
ATUL.NS,0.029209
BALKRISIND.NS,0.034501
BRITANNIA.NS,0.159590
CAPLIPOINT.NS,0.014584
COROMANDEL.NS,0.035237
DEEPAKNTR.NS,0.012673
FACT.NS,0.009898


121576.23785704061
-------------------------------


In [36]:
def main(start,end,future_start,future_end,money,year_1,year_2):
    start = start
    end = end

    breakout_window=10
    RSI_divergence_window = 30


    lst,b,closing_dataset = part_1(cd,bse_ticker,start=start,end=end,breakout_window=breakout_window,RSI_divergence_window=RSI_divergence_window)



    if len(lst) == 0:
        print("RSI NOT FOUND")
    else:
        filtered_ticker = part_2(lst,b,year_1,year_2,para,para_s)

        w = part_4(filtered_ticker,closing_dataset)
        display(w)

        future_start = future_start
        future_end= future_end
        # future_index = pd.date_range(start=future_start,end=future_end)
        money = part_5(cd,future_start,future_end,w,filtered_ticker,m)
    return money

In [ ]:
start_list = pd.date_range(start='2020-04-01',end='2022-04-01',freq='M')
m = 100000
year_1 = 'Dec 2020'
year_2 = 'Mar 2021'
for i in range(0,len(start_list)):
    start = start_list[i]
    end = start_list[i+12]
    future_start = start_list[i+12]
    future_end= start_list[i+13]
    print('Start:',start)
    print("End:",end)
    print("Future Start",future_start)
    print("Future End",future_end)
    m = main(start,end,future_start,future_end,m,year_1,year_2)
    print("-------------------------------")


Start: 2020-04-30 00:00:00
End: 2021-04-30 00:00:00
Future Start 2021-04-30 00:00:00
Future End 2021-05-31 00:00:00


,SMA,RSI,divergence,SMA_breakout,RSI_breakout,breakout,Industry
BAJAJ-AUTO.NS,-10.525820,0.772644,True,27.001581,1.628192,True,Automobile and Auto Components
BAJAJFINSV.NS,-21.269661,0.155133,True,95.758145,1.583615,True,Financial Services
HDFC.NS,-4.545881,0.444055,True,0.609216,0.131995,True,Financial Services
HIKAL.NS,-0.045259,0.849093,True,9.091878,1.162378,True,Healthcare
ICICIPRULI.NS,-1.516549,0.302719,True,9.926183,0.874729,True,Financial Services
ICIL.NS,-0.083132,0.094184,True,0.680303,0.119129,True,Textiles
INDIAMART.NS,-18.591793,0.498611,True,66.622202,0.826405,True,Consumer Services
JSL.NS,-0.069664,0.428593,True,1.453273,0.936948,True,Metals & Mining
JUBLPHARMA.NS,-3.210705,0.797402,True,7.940121,0.259677,True,Healthcare
JYOTHYLAB.NS,-0.320534,0.065439,True,1.089636,0.512627,True,Fast Moving Consumer Goods


Unnamed: 0,Year,Sales,Expenses,Operating Profit,OPM,Other_Income,Interest,Depreciation,Profit before tax,Tax %,Net_Profit,EPS in Rs,Dividend Payout %,Share_Capital,Reserves,Borrowings,Other_Liabilities,Total Liabilities,Fixed_Assets,CWIP,Investments,Other_Assets,Total Assets,CFO,CFI,CFF,Net Cash Flow,Company,Revenue,Financing Profit,Financing Margin %,Industry
10,Mar 2021,27741.0,22800.00,4942.00,18.0,1263.00,7.00,259.0,5939.00,23.0,4555.00,157.40,89.0,289.00,24913.00,121,6206.00,31530.00,1665.00,16,22631.00,7218.00,31530.00,3114.00,-2867.00,-20,228.00,BAJAJ-AUTO.NS,NaN,NaN,0,Automobile and Auto Components
10,Mar 2021,392.0,142.00,250.00,64.0,-0.00,0.00,5.0,245.00,27.0,179.00,11.23,27.0,80.00,3721.00,0,68.00,3869.00,159.00,2,3495.00,213.00,3869.00,169.00,-149.00,0,20.00,BAJAJFINSV.NS,NaN,NaN,0,Financial Services
10,Mar 2021,NaN,4587.00,NaN,0.0,26.00,28615.00,159.0,14815.00,19.0,12027.00,66.67,34.0,361.00,108115.00,441365,17758.00,567599.00,2196.00,0,68637.00,496765.00,567599.00,-28473.00,-8500.00,34601,-2372.00,HDFC.NS,48150.0,14948.0,31,Financial Services
11,Mar 2021,1720.0,1398.00,323.00,19.0,5.00,36.00,85.0,206.00,36.0,133.00,10.80,19.0,25.00,909.00,610,370.00,1913.00,713.00,254,1.00,946.00,1913.00,229.00,-156.00,-97,-24.00,HIKAL.NS,NaN,NaN,0,Healthcare
10,Mar 2021,83180.0,83509.00,-330.00,-0.0,1668.00,56.00,60.0,1223.00,22.0,960.00,6.69,30.0,1436.00,7683.00,1200,206909.00,217228.00,435.00,22,212212.00,4559.00,217228.00,7320.00,-5090.00,1205,3434.00,ICICIPRULI.NS,NaN,NaN,0,Financial Services
11,Mar 2021,2515.0,2131.00,384.00,15.0,36.00,27.00,40.0,353.00,26.0,260.00,13.18,11.0,39.00,1235.00,541,431.00,2247.00,535.00,8,192.00,1512.00,2247.00,27.00,-196.00,158,-11.00,ICIL.NS,NaN,NaN,0,Textiles
10,Mar 2021,665.0,332.00,333.00,50.0,85.00,7.00,16.0,396.00,28.0,287.00,94.43,16.0,30.00,1586.00,63,835.00,2516.00,65.00,0,2267.00,184.00,2516.00,326.00,-1343.00,1038,22.00,INDIAMART.NS,NaN,NaN,0,Consumer Services
10,Mar 2021,11679.0,10283.00,1396.00,12.0,142.00,464.00,373.0,700.00,39.0,428.00,8.78,0.0,97.00,3002.00,2899,4266.00,10265.00,5679.00,55,491.00,4040.00,10265.00,1295.00,-154.00,-1107,34.00,JSL.NS,NaN,NaN,0,Metals & Mining
11,Mar 2021,2710.0,2282.00,427.00,16.0,48.00,102.00,100.0,273.00,21.0,214.00,13.46,37.0,16.00,1267.00,504,58.00,1845.00,127.00,0,1646.00,71.00,1845.00,608.00,117.00,-782,-56.00,JUBLPHARMA.NS,NaN,NaN,0,Healthcare
11,Mar 2021,1885.0,1568.00,317.00,17.0,-4.00,12.00,78.0,223.00,15.0,190.00,5.18,77.0,37.00,897.00,50,440.00,1424.00,457.00,10,141.00,815.00,1424.00,386.00,-127.00,-196,63.00,JYOTHYLAB.NS,NaN,NaN,0,Fast Moving Consumer Goods


,weights
BAJAJ-AUTO.NS,0.104867
INDIAMART.NS,0.044752
JSL.NS,0.033076
JYOTHYLAB.NS,0.126875
MCDOWELL-N.NS,0.122886
NAUKRI.NS,0.074764
PNCINFRA.NS,0.060954
SHILPAMED.NS,0.054286
SKFINDIA.NS,0.103589
SUNTV.NS,0.067927


112269.58354916045
-------------------------------
Start: 2020-05-31 00:00:00
End: 2021-05-31 00:00:00
Future Start 2021-05-31 00:00:00
Future End 2021-06-30 00:00:00


,SMA,RSI,divergence,SMA_breakout,RSI_breakout,breakout,Industry
ABB.NS,-0.449684,0.801833,True,15.887514,2.887346,True,Capital Goods
ABFRL.NS,-0.691212,1.322342,True,0.219333,0.910353,True,Consumer Services
APLAPOLLO.NS,-0.168453,0.001953,True,1.933849,1.442818,True,Capital Goods
APOLLOTYRE.NS,-0.318216,0.662812,True,0.958607,0.334863,True,Automobile and Auto Components
ASHOKLEY.NS,-0.052621,0.338455,True,1.062303,0.629787,True,Capital Goods
...,...,...,...,...,...,...,...
VIPIND.NS,-0.445626,1.805181,True,3.327879,2.618470,True,Consumer Durables
WESTLIFE.NS,-0.400260,2.213294,True,4.891939,1.170206,True,Consumer Services
WHIRLPOOL.NS,-1.775428,1.085411,True,9.682560,2.735946,True,Consumer Durables
ZEEL.NS,-0.671106,0.283910,True,2.016546,3.171495,True,Media Entertainment & Publication
